In [1]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

In [2]:
model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-sid")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-sid")

C:\Users\Gebruiker\anaconda3\Lib\site-packages\transformers\configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at superb/wav2vec2-base-superb-sid were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you

In [45]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
[dataset[i]["audio"]["array"] for i in range(2)]

Found cached dataset librispeech_asr_demo (C:/Users/Gebruiker/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


[array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
        0.0010376 ]),
 array([-1.52587891e-04, -9.15527344e-05, -1.83105469e-04, ...,
         9.76562500e-04,  9.46044922e-04, -4.88281250e-04])]

## Wav2Vec2 ##

In [89]:
import torch
import librosa
import pandas as pd
from torch.cuda.amp import autocast
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor, WavLMForXVector


# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Load your dataframe
# Assume df has columns: "file1", "file2", "label"
df = pd.read_csv('commonvoice-small-random.csv')

def load_audio(file_path, duration=5):
    speech, _ = librosa.load(file_path, sr=16000, mono=True, duration=duration)
    return speech

# Load pre-trained model and feature extractor for binary classification
model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-sid").to(device)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-sid")

# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv').to(device)
# model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv')

# Adjust the model for binary classification
model.num_labels = 2
model.config.num_labels = 2
model.config.id2label = {0: False, 1: True}
model.config.label2id = {False: 0, True: 1}

# Process the audio files in pairs
def process_pairs(df):
    speech1 = []
    speech2 = []
    labels = []
    
    for index, row in df.iterrows():
        audio1 = load_audio(row['file1'])
        audio2 = load_audio(row['file2'])
        speech1.append(audio1)
        speech2.append(audio2)
        labels.append(row['label'])
    
    return speech1, speech2, labels

speech1, speech2, labels = process_pairs(df)

def batch_process(speech1, speech2, labels, batch_size=1):
    num_batches = len(speech1) // batch_size + (1 if len(speech1) % batch_size != 0 else 0)
    predicted_labels = []
    scores = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(speech1))

        batch_speech1 = speech1[start_idx:end_idx]
        batch_speech2 = speech2[start_idx:end_idx]
        batch_labels = labels[start_idx:end_idx]

        # Feature extraction
        inputs1 = feature_extractor(batch_speech1, sampling_rate=16000, padding=True, return_tensors="pt")
        inputs2 = feature_extractor(batch_speech2, sampling_rate=16000, padding=True, return_tensors="pt")

        # Move tensors to GPU
        inputs1 = {key: value.to(device) for key, value in inputs1.items()}
        inputs2 = {key: value.to(device) for key, value in inputs2.items()}

        # Compute embeddings with mixed precision
        with torch.no_grad():
            with autocast():
                embeddings1 = model(**inputs1).logits
                embeddings2 = model(**inputs2).logits

        # Clear cache to free up memory
        torch.cuda.empty_cache()

        # Compute similarity (e.g., cosine similarity) and classify
        similarities = torch.nn.functional.cosine_similarity(embeddings1, embeddings2)
        predictions = (similarities > 0.5).long()  # Threshold can be adjusted

        # Convert predictions to labels
        batch_predicted_labels = [model.config.id2label[pred.item()] for pred in predictions]
        predicted_labels.extend(batch_predicted_labels)
        scores.extend(similarities)

    return predicted_labels, scores


predicted_labels, scores = batch_process(speech1, speech2, labels, batch_size=1)

# Print results
for i, pred_label in enumerate(predicted_labels):
    print(f"Pair {i+1}: Predicted - {pred_label}, Actual - {labels[i]}")


Using device: cuda


FileNotFoundError: [Errno 2] No such file or directory: 'commonvoice-novowels.csv'

## WavLM ##

In [85]:
import torch
import librosa
import torchaudio
import pandas as pd
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Load your dataframe
# Assume df has columns: "file1", "file2", "label"
df = pd.read_csv('commonvoice-medium.csv')

resampler = torchaudio.transforms.Resample(orig_freq=16000, new_freq=16000)


def load_audio(file_path, duration=30):
    speech, _ = librosa.load(file_path, sr=16000, mono=True, duration=duration)
    return speech

# Load pre-trained model and feature extractor for speaker verification
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv')
model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv').to(device)

# Process the audio files in pairs
def process_pairs(df):
    speech1 = []
    speech2 = []
    labels = []
    
    for index, row in df.iterrows():
        audio1 = load_audio(row['file1'], 30)
        audio2 = load_audio(row['file2'], 30)
        speech1.append(audio1)
        speech2.append(audio2)
        labels.append(row['label'])
    
    return speech1, speech2, labels

speech1, speech2, labels = process_pairs(df)

def batch_process(speech1, speech2, labels, batch_size=1, threshold=0.5):
    num_batches = len(speech1) // batch_size + (1 if len(speech1) % batch_size != 0 else 0)
    predicted_labels = []
    scores = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(speech1))

        batch_speech1 = speech1[start_idx:end_idx]
        batch_speech2 = speech2[start_idx:end_idx]
        batch_labels = labels[start_idx:end_idx]

        # Feature extraction
        inputs1 = feature_extractor(batch_speech1, sampling_rate=16000, padding=True, return_tensors="pt")
        inputs2 = feature_extractor(batch_speech2, sampling_rate=16000, padding=True, return_tensors="pt")

        # Move tensors to GPU
        inputs1 = {key: value.to(device) for key, value in inputs1.items()}
        inputs2 = {key: value.to(device) for key, value in inputs2.items()}

        # Compute embeddings
        with torch.no_grad():
            embeddings1 = model(**inputs1).embeddings
            embeddings2 = model(**inputs2).embeddings

        embeddings1 = torch.nn.functional.normalize(embeddings1, dim=-1).cpu()
        embeddings2 = torch.nn.functional.normalize(embeddings2, dim=-1).cpu()

        # Compute cosine similarity and classify
        cosine_sim = torch.nn.CosineSimilarity(dim=-1)
        similarities = cosine_sim(embeddings1, embeddings2)
        predictions = (similarities > threshold).long()  # Threshold can be adjusted

        # Convert predictions to labels
        batch_predicted_labels = [True if pred == 1 else False for pred in predictions]
        predicted_labels.extend(batch_predicted_labels)
        
        scores.extend(similarities)
        
        print("done with batch")


        # Clear cache to free up memory
        torch.cuda.empty_cache()

    return predicted_labels, scores

predicted_labels, scores = batch_process(speech1, speech2, labels, batch_size=1, threshold=0.86)

Using device: cuda


Some weights of the model checkpoint at microsoft/wavlm-base-sv were not used when initializing WavLMForXVector: ['wavlm.encoder.pos_conv_embed.conv.weight_g', 'wavlm.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForXVector were not initialized from the model checkpoint at microsoft/wavlm-base-sv and are newly initialized: ['wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done wit

done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done wit

done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done wit

done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done with batch
done wit

In [107]:
import pandas as pd
import torchaudio
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
import torch

# Load the CSV file
csv_file_path = 'commonvoice-medium.csv'
data = pd.read_csv(csv_file_path)

# Load the feature extractor and the model
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv')
model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv')

# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

resampler = torchaudio.transforms.Resample(orig_freq=22500, new_freq=16000)

# Define a function to load and preprocess the audio files
def load_audio(filepath, max_duration):
    waveform, sample_rate = torchaudio.load(filepath)
    if sample_rate != 16000:
        waveform = resampler(waveform)
    if max_duration is not None:
        max_samples = int(max_duration * 16000)  # 16000 samples per second
        waveform = waveform[:, :max_samples]
    return waveform.squeeze().numpy(), 16000

# Process audio files in batches
batch_size = 1
pred_labels = []
scores = []
true_labels = []
threshold = 0.5  # Set your threshold here

for i in range(0, len(data), batch_size):
    batch = data.iloc[i:i+batch_size]
    
    audio1_list, audio2_list, labels = [], [], []
    
    for index, row in batch.iterrows():
        filepath1 = row['file1']
        filepath2 = row['file2']
        label = row['label']
        
        # Load and preprocess the audio files
        audio1, sr1 = load_audio(filepath1, 30)
        audio2, sr2 = load_audio(filepath2, 30)
        
        # Ensure the sample rates are compatible with the feature extractor
        assert sr1 == feature_extractor.sampling_rate, f"Sample rate mismatch: {sr1} vs {feature_extractor.sampling_rate}"
        assert sr2 == feature_extractor.sampling_rate, f"Sample rate mismatch: {sr2} vs {feature_extractor.sampling_rate}"
        
        audio1_list.append(audio1)
        audio2_list.append(audio2)
        labels.append(label)
    
    # Extract features and compute embeddings for both sets of audio files
    inputs1 = feature_extractor(audio1_list, return_tensors="pt", sampling_rate=feature_extractor.sampling_rate, padding=True).to(device)
    inputs2 = feature_extractor(audio2_list, return_tensors="pt", sampling_rate=feature_extractor.sampling_rate, padding=True).to(device)
    
    with torch.no_grad():
        embeddings1 = model(**inputs1).embeddings
        embeddings2 = model(**inputs2).embeddings
    
    embeddings1 = torch.nn.functional.normalize(embeddings1, dim=-1).cpu()
    embeddings2 = torch.nn.functional.normalize(embeddings2, dim=-1).cpu()
    
    # Compute cosine similarity
    cosine_sim = torch.nn.CosineSimilarity(dim=-1)
    similarities = cosine_sim(embeddings1, embeddings2)
    
    # Determine if the speakers are the same based on the threshold
    predicted_labels = (similarities >= threshold).int().tolist()
    
    # Store the results
    scores.extend(similarities)
    pred_labels.extend(predicted_labels)
    true_labels.extend(labels)
    
    print("done with batch " + str(batch))



Some weights of the model checkpoint at microsoft/wavlm-base-sv were not used when initializing WavLMForXVector: ['wavlm.encoder.pos_conv_embed.conv.weight_g', 'wavlm.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForXVector were not initialized from the model checkpoint at microsoft/wavlm-base-sv and are newly initialized: ['wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

done with batch    Unnamed: 0                                              file1  \
0           0  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                               file2  label  ID  
0  commonvoice-nl/commonvoice-nl/nl/clips/common_...   True   0  
done with batch    Unnamed: 0                                              file1  \
1           1  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                               file2  label  ID  
1  commonvoice-nl/commonvoice-nl/nl/clips/common_...   True   1  
done with batch    Unnamed: 0                                              file1  \
2           2  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                               file2  label  ID  
2  commonvoice-nl/commonvoice-nl/nl/clips/common_...   True   2  
done with batch    Unnamed: 0                                              file1  \
3           3  commonvoice-nl/commonvoice-nl/nl/clips/common_

C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type 

done with batch    Unnamed: 0                                              file1  \
4           4  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                               file2  label  ID  
4  commonvoice-nl/commonvoice-nl/nl/clips/common_...   True   4  
done with batch    Unnamed: 0                                              file1  \
5           5  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                               file2  label  ID  
5  commonvoice-nl/commonvoice-nl/nl/clips/common_...   True   5  
done with batch    Unnamed: 0                                              file1  \
6           6  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                               file2  label  ID  
6  commonvoice-nl/commonvoice-nl/nl/clips/common_...   True   6  
done with batch    Unnamed: 0                                              file1  \
7           7  commonvoice-nl/commonvoice-nl/nl/clips/common_

C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


...

C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


done with batch       Unnamed: 0                                              file1  \
1774        1774  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                                  file2  label    ID  
1774  commonvoice-nl/commonvoice-nl/nl/clips/common_...  False  1774  
done with batch       Unnamed: 0                                              file1  \
1775        1775  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                                  file2  label    ID  
1775  commonvoice-nl/commonvoice-nl/nl/clips/common_...  False  1775  
done with batch       Unnamed: 0                                              file1  \
1776        1776  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                                  file2  label    ID  
1776  commonvoice-nl/commonvoice-nl/nl/clips/common_...  False  1776  
done with batch       Unnamed: 0                                              file1  \
1777      

C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


done with batch       Unnamed: 0                                              file1  \
1778        1778  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                                  file2  label    ID  
1778  commonvoice-nl/commonvoice-nl/nl/clips/common_...  False  1778  
done with batch       Unnamed: 0                                              file1  \
1779        1779  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                                  file2  label    ID  
1779  commonvoice-nl/commonvoice-nl/nl/clips/common_...  False  1779  
done with batch       Unnamed: 0                                              file1  \
1780        1780  commonvoice-nl/commonvoice-nl/nl/clips/common_...   

                                                  file2  label    ID  
1780  commonvoice-nl/commonvoice-nl/nl/clips/common_...  False  1780  
done with batch       Unnamed: 0                                              file1  \
1781      

C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
C:\Users\Gebruiker\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [110]:
score = [(tensor.item()) for tensor in scores]
predicted_labels

[1]

In [109]:
import matplotlib.pyplot as plt
import seaborn as sns

# true_labels = [model.config.label2id[label] for label in labels]
# pred_labels = [model.config.label2id[label] for label in predicted_labels]

# Generate classification report
report = classification_report(labels, predicted_labels)
print("Classification Report:\n", report)

# Generate confusion matrix
conf_matrix = confusion_matrix(labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['false', 'true'], yticklabels=['false', 'true'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [1782, 1]

In [111]:
import pandas as pd
import numpy as np
import os
from operator import itemgetter
from scipy.optimize import brentq
from sklearn.metrics import roc_curve
from scipy.interpolate import interp1d

def calculate_eer(y, y_score, pos):
# y denotes groundtruth scores,
# y_score denotes the prediction scores.

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=pos)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)

    return eer, thresh, fpr, tpr

# Creates a list of false-negative rates, a list of false-positive rates
# and a list of decision thresholds that give those error-rates.
def ComputeErrorRates(scores, labels):

    # Sort the scores from smallest to largest, and also get the corresponding
    # indexes of the sorted scores.  We will treat the sorted scores as the
    # thresholds at which the the error-rates are evaluated.
    sorted_indexes, thresholds = zip(*sorted(
        [(index, threshold) for index, threshold in enumerate(scores)],
        key=itemgetter(1)))
    sorted_labels = []
    labels = [labels[i] for i in sorted_indexes]
    fnrs = []
    fprs = []

    # At the end of this loop, fnrs[i] is the number of errors made by
    # incorrectly rejecting scores less than thresholds[i]. And, fprs[i]
    # is the total number of times that we have correctly accepted scores
    # greater than thresholds[i].
    for i in range(0, len(labels)):
        if i == 0:
            fnrs.append(labels[i])
            fprs.append(1 - labels[i])
        else:
            fnrs.append(fnrs[i-1] + labels[i])
            fprs.append(fprs[i-1] + 1 - labels[i])
    fnrs_norm = sum(labels)
    fprs_norm = len(labels) - fnrs_norm

    # Now divide by the total number of false negative errors to
    # obtain the false positive rates across all thresholds
    fnrs = [x / float(fnrs_norm) for x in fnrs]

    # Divide by the total number of corret positives to get the
    # true positive rate.  Subtract these quantities from 1 to
    # get the false positive rates.
    fprs = [1 - x / float(fprs_norm) for x in fprs]
    return fnrs, fprs, thresholds

# Computes the minimum of the detection cost function.  The comments refer to
# equations in Section 3 of the NIST 2016 Speaker Recognition Evaluation Plan.
def ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa):
    min_c_det = float("inf")
    min_c_det_threshold = thresholds[0]
    for i in range(0, len(fnrs)):
        # See Equation (2).  it is a weighted sum of false negative
        # and false positive errors.
        c_det = c_miss * fnrs[i] * p_target + c_fa * fprs[i] * (1 - p_target)
        if c_det < min_c_det:
            min_c_det = c_det
            min_c_det_threshold = thresholds[i]
    # See Equations (3) and (4).  Now we normalize the cost.
    c_def = min(c_miss * p_target, c_fa * (1 - p_target))
    min_dcf = min_c_det / c_def
    return min_dcf, min_c_det_threshold

In [113]:
eer, thresh, fpr, tpr = calculate_eer(labels, score, pos=1)
print('EER : %.2f%%'%(eer*100))

EER : 27.72%


In [115]:
c_miss = 1
c_fa = 1
p_target = 0.01

fnrs, fprs, thresholds = ComputeErrorRates(score,
                                           labels)
mindcf, threshold_default = ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa)

print("minDCF : {0:.4f}, at threshold {1:.4f} (p-target={2}, c-miss={3}, "
    "c-fa={4})".format(mindcf, threshold_default, p_target,c_miss, c_fa))

minDCF : 0.9450, at threshold 0.9737 (p-target=0.01, c-miss=1, c-fa=1)
